In [ ]:
import torch
import torch.nn as nn
import torchvision
from torchvision import models

In [ ]:
class ImageEncoder(nn.Module):
    def __init__(self, backbone_name='resnet50', pretrained=True, hidden_dim=256):
        super(ImageEncoder, self).__init__()

        backbone = models.resnet50(pretrained=pretrained)
        self.backbone = nn.Sequential(*list(backbone.children())[:-2])
        
        self.hidden_dim = hidden_dim
        self.conv1x1 = nn.Conv2d(backbone.fc.in_features, hidden_dim, kernel_size=1)
    
    def forward(self, x):
        features = self.backbone(x)
        features = self.conv1x1(features)
        
        return features